In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob


In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

In [3]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_RAW_DIR = os.path.join(PROJECT_ROOT, 'data', 'raw')

print("Project_root:", PROJECT_ROOT)
print("Data_raw_dir:", DATA_RAW_DIR)

Project_root: c:\Users\arjun\aqi-forecasting-india-ml-dl
Data_raw_dir: c:\Users\arjun\aqi-forecasting-india-ml-dl\data\raw


In [4]:
csv_files = sorted(glob.glob(os.path.join(DATA_RAW_DIR, '*.csv')))
csv_files

['c:\\Users\\arjun\\aqi-forecasting-india-ml-dl\\data\\raw\\bengaluru_combined.csv',
 'c:\\Users\\arjun\\aqi-forecasting-india-ml-dl\\data\\raw\\delhi_combined.csv',
 'c:\\Users\\arjun\\aqi-forecasting-india-ml-dl\\data\\raw\\mumbai_combined.csv']

In [5]:
df=pd.read_csv(csv_files[0])
df.head()

,Timestamp,Location,PM2.5,PM10,NO2,NH3,SO2,CO,O3
0,01-01-2020,Bengaluru - Silk Board,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02-01-2020,Bengaluru - Silk Board,43.67,134.00,20.28,10.98,3.41,0.91,21.82
2,03-01-2020,Bengaluru - Silk Board,30.58,74.42,15.17,12.10,3.27,0.96,23.31
3,04-01-2020,Bengaluru - Silk Board,66.35,155.68,42.90,11.75,3.26,2.54,29.70
4,05-01-2020,Bengaluru - Silk Board,48.00,99.13,18.56,9.79,2.95,1.14,31.01


In [6]:
df.columns

Index(['Timestamp', 'Location', 'PM2.5', 'PM10', 'NO2', 'NH3', 'SO2', 'CO', 'O3'], dtype='object')

In [7]:
p_data_columns= [c for c in df.columns if 'data' in c.lower() or 'time' in c.lower()]
p_data_columns

['Timestamp']

In [8]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"], dayfirst=True, errors='coerce')

print("Row",len(df))
print("null timestamp:",df["Timestamp"].isnull().sum())

df = df.dropna(subset=["Timestamp"]).copy()
df = df.sort_values(by="Timestamp").reset_index(drop=True)

print("date range:", df["Timestamp"].min(), "to", df["Timestamp"].max())
df.head()

Row 1827
null timestamp: 0
date range: 2020-01-01 00:00:00 to 2024-12-31 00:00:00


,Timestamp,Location,PM2.5,PM10,NO2,NH3,SO2,CO,O3
0,2020-01-01,Bengaluru - Silk Board,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,Bengaluru - Silk Board,43.67,134.00,20.28,10.98,3.41,0.91,21.82
2,2020-01-03,Bengaluru - Silk Board,30.58,74.42,15.17,12.10,3.27,0.96,23.31
3,2020-01-04,Bengaluru - Silk Board,66.35,155.68,42.90,11.75,3.26,2.54,29.70
4,2020-01-05,Bengaluru - Silk Board,48.00,99.13,18.56,9.79,2.95,1.14,31.01


In [9]:
dups = df.duplicated(subset=["Timestamp"]).sum()
print("duplicate timestamps:", dups)

if dups > 0:
    df=df.groupby("Timestamp", as_index=False).mean(numeric_only=True)
    print("Current rows:", len(df))

duplicate timestamps: 0


In [10]:
df=df.set_index("Timestamp").sort_index()

full_index=pd.date_range(df.index.min(),df.index.max(), freq='D')
missing_dates=full_index.difference(df.index)

print("Expected daily length:", len(full_index))
print("Actual length:", len(df))
print("Missing dates:", len(missing_dates))
print("Missing day%",round(100*len(missing_dates)/len(full_index),2))

missing_dates[:20]

Expected daily length: 1827
Actual length: 1827
Missing dates: 0
Missing day% 0.0


DatetimeIndex([], dtype='datetime64[ns]', freq='D')

In [11]:
pollutant_cols = ["PM2.5", "PM10", "NO2", "NH3", "SO2", "CO", "O3"]

missing_by_col = df[pollutant_cols].isna().mean().sort_values(ascending=False) * 100
missing_by_col


SO2      8.319650
O3       7.443897
NO2      6.185003
NH3      6.130268
PM10     4.050356
CO       4.050356
PM2.5    3.995621
dtype: float64

In [12]:
def city_csv(filepath: str) -> pd.DataFrame:
    tmp=pd.read_csv(filepath)

    expected = ["Timestamp", "Location", "PM2.5", "PM10", "NO2", "NH3", "SO2", "CO", "O3"]
    missing_cols = [c for c in expected if c not in tmp.columns]
    if missing_cols:
        raise ValueError(f"{os.path.basename(filepath)} is missing columns: {missing_cols}")
    
    # Parse timestamps
    tmp["Timestamp"] = pd.to_datetime(tmp["Timestamp"], dayfirst=True, errors='coerce')
    tmp = tmp.dropna(subset=["Timestamp"]).copy()
    tmp = tmp.sort_values(by="Timestamp")
    
    # Handle duplicate timestamps by averaging
    if tmp.duplicated(subset=["Timestamp"]).any():
        tmp = tmp.groupby(["Timestamp", "Location"], as_index=False).mean(numeric_only=True)
    
    # Set index
    tmp = tmp.set_index("Timestamp").sort_index()

    #enforce daily grid
    full_in=pd.date_range(tmp.index.min(),tmp.index.max(), freq='D')
    tmp=tmp.reindex(full_in)

    #Timestamp back to column
    tmp.index.name = "Timestamp"
    tmp = tmp.reset_index()

    # Add City column based on filename
    tmp["City"] = os.path.splitext(os.path.basename(filepath))[0]
    return tmp

In [13]:
#Inspect all city files
city_com=[os.path.basename(f) for f in csv_files]
city_com

['bengaluru_combined.csv', 'delhi_combined.csv', 'mumbai_combined.csv']

In [14]:
City_name={"Delhi":'delhi_combined.csv',
           "Mumbai":'mumbai_combined.csv',
           "Bengaluru":'bengaluru_combined.csv',
           }

In [15]:
dfs=[]
for city, filename in City_name.items():
    path=os.path.join(DATA_RAW_DIR, filename)
    tmp=city_csv(path)

    tmp["City"]=city
    dfs.append(tmp)

df=pd.concat(dfs, ignore_index=True)
df.head()

,Timestamp,Location,PM2.5,PM10,NO2,NH3,SO2,CO,O3,City
0,2020-01-01,Delhi - Punjabi Bagh,420.68,507.60,105.21,63.96,6.30,2.26,9.49,Delhi
1,2020-01-02,Delhi - Punjabi Bagh,364.73,480.09,79.38,51.28,9.69,2.87,8.03,Delhi
2,2020-01-03,Delhi - Punjabi Bagh,227.72,309.23,57.22,44.29,11.04,2.69,8.18,Delhi
3,2020-01-04,Delhi - Punjabi Bagh,243.86,323.75,60.99,44.88,12.61,1.94,11.72,Delhi
4,2020-01-05,Delhi - Punjabi Bagh,161.66,222.79,56.22,38.35,11.03,1.91,12.37,Delhi


In [16]:
summary=(
    df.groupby("City")["Timestamp"]
    .agg(start='min',end='max',n_days='count')
    .reset_index()
)
summary

,City,start,end,n_days
0,Bengaluru,2020-01-01,2024-12-31,1827
1,Delhi,2020-01-01,2024-12-31,1827
2,Mumbai,2020-01-01,2024-12-31,1827


In [17]:
pol_col=["PM2.5", "PM10", "NO2", "NH3", "SO2", "CO", "O3"]

missing_summary=(
    df.groupby("City")[pol_col]
    .apply(lambda x: (x.isna().mean()*100).round(2))
    .reset_index()
)
missing_summary

,City,PM2.5,PM10,NO2,NH3,SO2,CO,O3
0,Bengaluru,4.00,4.05,6.19,6.13,8.32,4.05,7.44
1,Delhi,0.77,1.09,1.15,1.26,2.35,1.48,1.09
2,Mumbai,3.89,2.41,2.52,3.12,4.38,1.15,1.92


In [18]:
pollutant_cols = ["PM2.5", "PM10", "NO2", "NH3", "SO2", "CO", "O3"]

def longest_nan_run(s: pd.Series) -> int:
    is_nan = s.isna().to_numpy()
    longest = 0
    current = 0
    for f in is_nan:
        if f:
            current += 1
            longest = max(longest, current)
        else:
            current = 0
    return longest

runs = []
for city in df["City"].unique():
    tmp = df[df["City"] == city].sort_values("Timestamp")
    for col in pollutant_cols:
        runs.append({
            "City": city,
            "Pollutant": col,
            "Longest_Consecutive_NaNs": longest_nan_run(tmp[col])
        })

nan_run_summary = pd.DataFrame(runs).sort_values(["City", "Longest_Consecutive_NaNs"], ascending=[True, False])
nan_run_summary


,City,Pollutant,Longest_Consecutive_NaNs
18,Bengaluru,SO2,78
20,Bengaluru,O3,50
16,Bengaluru,NO2,33
17,Bengaluru,NH3,33
14,Bengaluru,PM2.5,15
15,Bengaluru,PM10,15
19,Bengaluru,CO,15
0,Delhi,PM2.5,6
1,Delhi,PM10,6
2,Delhi,NO2,6
